In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from apiclient.http import MediaIoBaseDownload
from googleapiclient.discovery import build
import pickle
from google.cloud import storage
gcs_service = build('storage', 'v1')

In [ ]:
#read csv from bucket

with open('/tmp/gsutil_download.csv', 'wb') as f:
  # Download the file from a given Google Cloud Storage bucket.
  request = gcs_service.objects().get_media(bucket='malmem-dataset',
                                            object='malmem_dataset.csv')
  media = MediaIoBaseDownload(f, request)

  done = False
  while not done:
    _, done = media.next_chunk()

df = pd.read_csv('/tmp/gsutil_download.csv')

In [ ]:
X = df.copy()

In [ ]:
X = X.drop('Category', axis=1)
class_map = {'Benign':0, 'Malware':1}
X['Class'] = X['Class'].map(class_map)
Y = X['Class']
X = X.drop('Class', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

In [ ]:
bucket_name = 'malmem-dataset'
blob_name = 'rf_model.pkl'

storage_client = storage.Client()
bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(blob_name)

blob.upload_from_string(pickle.dumps(rf))